In [1]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '${:,.2f}'.format

In [2]:
csv_path = 'Resources/purchase_data.csv'
data_df = pd.read_csv(csv_path)

In [3]:
# Number of Players

num_players = data_df["SN"].nunique()

players_df = pd.DataFrame({"Number of Players": [num_players]})

In [4]:
# Purchasing Analysis (Total)

purchasing_data = {
    "Number of Unique Items": [data_df["Item Name"].nunique()],
    "Average Price": [round(data_df["Price"].mean(), 2)],
    "Number of Purchases": [data_df["Purchase ID"].count()],
    "Total Revenue": [data_df["Price"].sum()]
}

purchasing_df = pd.DataFrame(purchasing_data)

In [5]:
# Gender Demographics

gender_gpby = data_df.groupby("Gender")

gender_tally_df = pd.DataFrame(gender_gpby["SN"].nunique())
gender_tally_df = gender_tally_df.rename(columns={"SN": "Total Count"})
gender_tally_df["Percentage of Players"] = (100 * gender_tally_df["Total Count"] / num_players).map("{:.2f}%".format)

In [6]:
# Purchasing Analysis (Gender)

gender_analysis_df = pd.DataFrame()

gender_analysis_df["Purchase Count"] = gender_gpby["Purchase ID"].count()
gender_analysis_df["Average Purchase Price"] = gender_gpby["Price"].mean()
gender_analysis_df["Total Purchase Value"] = gender_gpby["Price"].sum()
gender_analysis_df["Avg Total Purchase per Person"] = gender_analysis_df["Total Purchase Value"] / gender_tally_df["Total Count"]

In [7]:
# Age Demographics

age_bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 1000]
bin_names = ["< 10", "10 - 14", "15 - 19", "20 - 24", "25 - 29", "30 - 34", "35 - 39", "> 40"]

data_df["Age Range"] = pd.cut(data_df["Age"], age_bins, labels=bin_names, include_lowest=True)

age_gpby = data_df.groupby("Age Range")

age_tally_df = pd.DataFrame(age_gpby["SN"].nunique())
age_tally_df = age_tally_df.rename(columns={"SN": "Total Count"})
age_tally_df["Percentage of Players"] = (100 * age_tally_df["Total Count"] / num_players).map("{:.2f}%".format)

In [8]:
# Purchasing Analysis (Age)

age_analysis_df = pd.DataFrame()

age_analysis_df["Purchase Count"] = age_gpby["Purchase ID"].count()
age_analysis_df["Average Purchase Price"] = age_gpby["Price"].mean()
age_analysis_df["Total Purchase Value"] = age_gpby["Price"].sum()
age_analysis_df["Avg Total Purchase per Person"] = age_analysis_df["Total Purchase Value"] / age_tally_df["Total Count"]

In [9]:
# Top Spenders

players_gpby = data_df.groupby("SN")

# get the 5 highest spenders & their data, then sort & get top 5 using iloc
top_spenders_df = players_gpby["Price"].sum().reset_index()
top_spenders_df["Purchase Count"] = players_gpby["Purchase ID"].count().reset_index()["Purchase ID"]
top_spenders_df["Average Purchase Price"] = top_spenders_df["Price"] / top_spenders_df["Purchase Count"]
top_spenders_df = top_spenders_df.sort_values("Price", ascending=False).iloc[:5]

# do housekeeping
top_spenders_df = top_spenders_df.set_index("SN")
top_spenders_df = top_spenders_df.rename(columns={"Price": "Total Purchase Value"})
top_spenders_df = top_spenders_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

In [10]:
# Most Popular Items

most_pop_gpby = data_df[["Item ID", "Item Name", "Price", "Purchase ID"]].groupby(["Item ID", "Item Name"])

# get the 5 most frequently purchased items & their data
by_item_df = most_pop_gpby["Purchase ID"].count().reset_index()
by_item_df["Item Price"] = most_pop_gpby["Price"].mean().reset_index()["Price"]
by_item_df["Total Purchase Value"] = most_pop_gpby["Price"].sum().reset_index()["Price"]

# do housekeeping
by_item_df = by_item_df.rename(columns={"Purchase ID": "Purchase Count"})
by_item_df = by_item_df.set_index("Item ID")

# sort & get top 5 using iloc
most_pop_df = by_item_df.sort_values("Purchase Count", ascending=False).iloc[:5]

In [11]:
# Most Profitable Items

# already have the table from most popular, just need to sort by total value instead
most_profitable_df = by_item_df.sort_values("Total Purchase Value", ascending=False).iloc[:5]

In [12]:
# Output some HTML

# HTML5 boilerplate
html_string = "let text = `"
html_string += f'<div id="players" class="container table-container">{players_df.to_html(index=False)}</div>'
html_string += f'<div id="purchasing" class="container table-container">{purchasing_df.to_html(index=False)}</div>'
html_string += f'<div id="genderDemos" class="container table-container">{gender_tally_df.reset_index().to_html(index=False)}</div>'
html_string += f'<div id="genderAnalysis" class="container table-container">{gender_analysis_df.reset_index().to_html(index=False)}</div>'
html_string += f'<div id="ageDemos" class="container table-container">{age_tally_df.reset_index().to_html(index=False)}</div>'
html_string += f'<div id="ageAnalysis" class="container table-container">{age_analysis_df.reset_index().to_html(index=False)}</div>'
html_string += f'<div id="topSpenders" class="container table-container">{top_spenders_df.reset_index().to_html(index=False)}</div>'
html_string += f'<div id="popularItems" class="container table-container">{most_pop_df.reset_index().to_html(index=False)}</div>'
html_string += f'<div id="profitableItems" class="container table-container">{most_profitable_df.reset_index().to_html(index=False)}</div>'
html_string += "`"

In [13]:
with open('Output/output.txt', 'w') as f:
    f.truncate(0)
    f.write(html_string)